<a href="https://colab.research.google.com/github/gumberankush/AI-DataScience-Course/blob/main/Deep%20Learning/Keras_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference: https://www.tensorflow.org/guide/keras/functional
# Keras

Keras is a high-level neural networks API, written in Python and capable of running on top of TensorFlow, CNTK, or Theano. It was developed with a focus on enabling fast experimentation. Being able to go from idea to result with the least possible delay is key to doing good research.

Use Keras if you need a deep learning library that:



*   Allows for easy and fast prototyping (through user friendliness, modularity, and extensibility).
*   Supports both convolutional networks and recurrent networks, as well as combinations of the two.
*   Runs seamlessly on CPU and GPU.

## Keras backends

What is a "backend"?

Keras is a model-level library, providing high-level building blocks for developing deep learning models. It does not handle itself low-level operations such as tensor products, convolutions and so on. Instead, it relies on a specialized, well-optimized tensor manipulation library to do so, serving as the "backend engine" of Keras. Rather than picking one single tensor library and making the implementation of Keras tied to that library, Keras handles the problem in a modular way, and several different backend engines can be plugged seamlessly into Keras.

At this time, Keras has three backend implementations available: **the TensorFlow backend, the Theano backend, and the CNTK backend**.

TensorFlow is an open-source symbolic tensor manipulation framework developed by Google.

Theano is an open-source symbolic tensor manipulation framework developed by LISA Lab at Université de Montréal. ( No further development on this framework)

CNTK is an open-source toolkit for deep learning developed by Microsoft.

In [2]:
import tensorflow as tf
import tensorflow.keras

## Layers

A Layer encapsulates a state (weights) and some computation (defined in the call method).

In [ ]:
tf.keras.layers.Layer(
    trainable=True, name=None, dtype=None, dynamic=False, **kwargs
)

A layer is a class implementing common neural networks operations, such as convolution, batch norm, etc. These operations require managing weights, losses, updates, and inter-layer connectivity.

Users will just instantiate a layer and then treat it as a callable.

It is recommended that descendants of Layer implement the following methods:

**__init__()**: Save configuration in member variables

**build()**: Called once from __call__, when we know the shapes of inputs and dtype. Should have the calls to add_weight(), and then call the super's build() (which sets self.built = True, which is nice in case the user wants to call build() manually before the first __call__).

**call()**: Called in __call__ after making sure build() has been called once. Should actually perform the logic of applying the layer to the input tensors (which should be passed in as the first argument).

In [5]:
from tensorflow.keras.layers import Layer

class Linear(Layer):
  """y = w.x + b"""

  def __init__(self, units=32, input_dim=32): # constructor
      super(Linear, self).__init__()
      w_init = tf.random_normal_initializer() # weight's random normal intialization
      self.w = tf.Variable(
          initial_value=w_init(shape=(input_dim, units), dtype='float32'),
          trainable=True)
      b_init = tf.zeros_initializer() # bias intialized with Zero
      self.b = tf.Variable(
          initial_value=b_init(shape=(units,), dtype='float32'),
          trainable=True)

  def call(self, inputs):
      return tf.matmul(inputs, self.w) + self.b

### Model 
The core data structure of Keras is a model, a way to organize layers. The simplest type of model is the **Sequential model**, a linear stack of layers. For more complex architectures, one should use the Keras functional API, which allows to build arbitrary graphs of layers.

Example of Sequential Model is as folows

In [6]:
from tensorflow.keras import Sequential

model = Sequential()

## Built-in layers

Keras provides us with a [wide range of built-in layers](https://www.tensorflow.org/api_docs/python/tf/keras/layers/), so that you don't have to implement your own layers all the time.
- Dense layers
- Convolution layers
- Transposed convolutions
- Separateable convolutions
- Average and max pooling
- Global average and max pooling
- LSTM, GRU (with built-in cuDNN acceleration)
- BatchNormalization
- Dropout
- Attention
- ConvLSTM2D


Stacking is done through add()

In [ ]:
from tensorflow.keras.layers import Dense
# Optionally, the first layer can receive an `input_shape` argument:
model = Sequential()
model.add(Dense(32, input_shape=(500,)))
# Afterwards, we do automatic shape inference:
model.add(Dense(32))

# This is identical to the following:
model = Sequential()
model.add(Dense(32, input_dim=500))

# And to the following:
model = Sequential()
model.add(Dense(32, batch_input_shape=(None, 500)))

# Note that you can also omit the `input_shape` argument:
# In that case the model gets built the first time you call `fit` (or other
# training and evaluation methods).
model = Sequential()
model.add(Dense(32))  # layer 1
model.add(Dense(32))  # layer 2
model.compile(optimizer='sgd', loss=tf.keras.losses.MeanSquaredError()) # can provide any kind of optimizer and loss
# This builds the model for the first time:
model.fit(x, y, batch_size=32, epochs=10) #training

# Note that when using this delayed-build pattern (no input shape specified),
# the model doesn't have any weights until the first call
# to a training/evaluation method (since it isn't yet built):
model = Sequential()
model.add(Dense(32))
model.add(Dense(32))
model.weights  # returns []

# Whereas if you specify the input shape, the model gets built continuously
# as you are adding layers:
model = Sequential()
model.add(Dense(32, input_shape=(500,)))
model.add(Dense(32))
model.weights  # returns list of length 4

# When using the delayed-build pattern (no input shape specified), you can
# choose to manually build your model by calling `build(batch_input_shape)`:
model = Sequential()
model.add(Dense(32))
model.add(Dense(32))
model.build((None, 500))
model.weights  # returns list of length 4

Once the model is built let's configure the model's learning process with compile()

In [ ]:
compile(
    optimizer='rmsprop', loss=None, metrics=None, loss_weights=None,
    sample_weight_mode=None, weighted_metrics=None, target_tensors=None,
    distribute=None, **kwargs)

**optimizer**: String (name of optimizer) or optimizer instance. See tf.keras.optimizers.

**loss**: String (name of objective function), objective function or tf.keras.losses.Loss instance. See tf.keras.losses. An objective function is any callable with the signature scalar_loss = fn(y_true, y_pred). If the model has multiple outputs, you can use a different loss on each output by passing a dictionary or a list of losses. The loss value that will be minimized by the model will then be the sum of all individual losses.

**metrics**: List of metrics to be evaluated by the model during training and testing. Typically you will use metrics=['accuracy']. To specify different metrics for different outputs of a multi-output model, you could also pass a dictionary, such as metrics={'output_a': 'accuracy', 'output_b': ['accuracy', 'mse']}. You can also pass a list (len = len(outputs)) of lists of metrics such as metrics=[['accuracy'], ['accuracy', 'mse']] or metrics=['accuracy', ['accuracy', 'mse']].

**evaluate()**

Returns the loss value & metrics values for the model in test mode.

Computation is done in batches.

In [ ]:
evaluate(
    x=None, y=None, batch_size=None, verbose=1, sample_weight=None, steps=None,
    callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False
)

**fit()**

Trains the model for a fixed number of epochs (iterations on a dataset).

In [ ]:
fit(
    x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None,
    validation_split=0.0, validation_data=None, shuffle=True, class_weight=None,
    sample_weight=None, initial_epoch=0, steps_per_epoch=None,
    validation_steps=None, validation_freq=1, max_queue_size=10, workers=1,
    use_multiprocessing=False, **kwargs
)

We can now iterate on your training data in batches:

In [ ]:
# dummy code
# x_train and y_train are Numpy arrays --just like in the Scikit-Learn API.
model.fit(x_train, y_train, epochs=5, batch_size=32)

Evaluate your performance in one line:

In [ ]:
loss_and_metrics = model.evaluate(x_test, y_test, batch_size=128)

Or generate predictions on new data:

In [ ]:
classes = model.predict(x_test, batch_size=128)